<a href="https://colab.research.google.com/github/SamudralaAjaykumarrr/Real-Time-Fraud-Detection/blob/main/Real_Time_Fraud_Detection_(IsolationForest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y numpy jax jaxlib opencv-python opencv-python-headless opencv-contrib-python shap websockets google-genai google-adk dataproc-spark-connect

!pip install numpy==1.26.4
!pip install websockets==11.0.3


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: jax 0.7.2
Uninstalling jax-0.7.2:
  Successfully uninstalled jax-0.7.2
Found existing installation: jaxlib 0.7.2
Uninstalling jaxlib-0.7.2:
  Successfully uninstalled jaxlib-0.7.2
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88
Found existing installation: opencv-contrib-python 4.12.0.88
Uninstalling opencv-contrib-python-4.12.0.88:
  Successfully uninstalled opencv-contrib-python-4.12.0.88
Found existing installation: shap 0.50.0
Uninstalling shap-0.50.0:
  Successfully uninstalled shap-0.50.0
Found existing installation: websockets 11.0.3
Uninstalling websockets-11.0.3:
  Successfully unins

  Using cached websockets-11.0.3-py3-none-any.whl.metadata (6.6 kB)
Using cached websockets-11.0.3-py3-none-any.whl (118 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
^C


In [1]:
!pip install scikit-learn pandas gradio==4.16.0


  Using cached websockets-11.0.3-py3-none-any.whl.metadata (6.6 kB)
Using cached websockets-11.0.3-py3-none-any.whl (118 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 11.0.3 which is incompatible.


In [2]:
import numpy, pandas
print("Numpy OK:", numpy.__version__)


Numpy OK: 1.26.4


In [3]:
# ---------------------------------------------------
# 1. Install dependencies (after environment cleanup)
# ---------------------------------------------------
!pip install -q scikit-learn pandas gradio==4.16.0

# ---------------------------------------------------
# 2. Imports
# ---------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import gradio as gr

# ---------------------------------------------------
# 3. Create simulated transaction dataset
# ---------------------------------------------------
rng = np.random.default_rng(42)

# Normal transactions
normal_data = rng.normal(loc=50, scale=10, size=(1000, 3))

# Fraud transactions (higher values, odd patterns)
fraud_data = rng.normal(loc=120, scale=30, size=(40, 3))

# Combine
data = np.vstack([normal_data, fraud_data])
labels = np.array([0]*1000 + [1]*40)

df = pd.DataFrame(data, columns=["amount", "time_gap", "device_score"])
df["label"] = labels

# ---------------------------------------------------
# 4. Scale features
# ---------------------------------------------------
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[["amount", "time_gap", "device_score"]])

# ---------------------------------------------------
# 5. Train IsolationForest
# ---------------------------------------------------
model = IsolationForest(contamination=0.03, random_state=42)
model.fit(scaled_features)

# ---------------------------------------------------
# 6. Prediction function
# ---------------------------------------------------
def predict_fraud(amount, time_gap, device_score):
    x = np.array([[amount, time_gap, device_score]])
    x_scaled = scaler.transform(x)

    pred = model.predict(x_scaled)[0]   # -1 = fraud, 1 = normal
    score = model.decision_function(x_scaled)[0]

    status = "🚨 Fraud Detected" if pred == -1 else "✅ Normal Transaction"

    return {
        "Status": status,
        "Model Score (lower = fraud)": float(score)
    }

# ---------------------------------------------------
# 7. Gradio UI
# ---------------------------------------------------
amount_in = gr.Number(label="Transaction Amount ($)")
gap_in = gr.Number(label="Time Gap Since Last Transaction (seconds)")
device_in = gr.Number(label="Device Trust Score (0–100)")

outputs = gr.JSON(label="Prediction Result")

demo = gr.Interface(
    fn=predict_fraud,
    inputs=[amount_in, gap_in, device_in],
    outputs=outputs,
    title="Real-Time Fraud Detection (IsolationForest)",
    description="Simulates a real-time fraud detector using anomaly detection."
)

demo.launch()


IMPORTANT: You are using gradio version 4.16.0, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fd9fd9f91a36a4ad36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
